In [12]:
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import pickle

In [5]:
market = Market()
speculation_db = ADatabase("bitcoin_speculation")

In [6]:
speculation_db.connect()
data = speculation_db.retrieve("pdr_data")
speculation_db.disconnect()

In [8]:
factors = [str(x) for x in range(14)]
factors.append("adjclose")
factors.append("d1")
factors.append("d2")
factors.append("d3")

In [13]:
sim = []
speculation_db.connect()
errors = []
training_year = 1
for training_year in [1,4]:
    for year in tqdm(range(start_year,end_year)):
        try:
            training_data = data[(data["year"]<year) & (data["year"]>=year-training_year)]
            training_data.dropna(inplace=True)
            training_data.reset_index(drop=True,inplace=True)
            prediction_data = data[data["year"]==year].dropna().reset_index()
            if training_data.index.size > 100 and prediction_data.index.size > 40:
                refined = {"X":training_data[factors],"y":training_data["y"]}
                models = m.regression(refined)
                for row in models.iterrows():
                    try:
                        model = row[1]["model"]
                        api = row[1]["api"]
                        score = row[1]["score"]
                        prediction_data[f"{api}_prediction"] = model.predict(prediction_data[factors])
                        prediction_data[f"{api}_score"] = score 
                    except Exception as e:
                        print(str(e))
                included_columns = ["date","ticker","training_year"]
                included_columns.extend([x for x in prediction_data.columns if "score" in x or "prediction" in x])
                prediction_data["training_year"] = training_year
                speculation_db.store("pdr_sim",prediction_data[included_columns])
            else:
                errors.append([year,training_data.index.size,prediction_data.index.size,"not_enough_stuff"])
        except Exception as e:
            errors.append([year,training_data.index.size,prediction_data.index.size,str(e)])
speculation_db.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.01s/it]


In [14]:
pd.DataFrame(errors)

""
